In [ ]:
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import matplotlib.pyplot
BATCH_SIZE = 50
NUM_BATCHES = 3000
TEST_ID = 100
VALID_SIZE = 2000
TOP_PATCHES = 12
VISUAL_FILTERS = 5

def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2dl1(x, W):
    return tf.nn.conv2d(x, filters=W, strides=[1, 2, 2, 1], padding='SAME')


def conv2dl2(x, W):
    return tf.nn.conv2d(x, filters=W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool2d(input=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def visualize_layer(W):
    fig = plt.figure()
    for i in range(25):
        ax = fig.add_subplot(5, 5, i + 1)
        ax.set_xticks(())
        ax.set_yticks(())
        ax.imshow(W[:, :, :, i].reshape(12, 12), cmap='Greys_r')
    fig.show()


def visualize_patches(H, x_test):
    sort_a = np.argsort(H, axis=None)
    idx = np.unravel_index(sort_a, H.shape)
    test_idx = np.zeros(TOP_PATCHES, dtype=np.int)
    filter_idx = np.zeros(TOP_PATCHES, dtype=np.int)
    activation_set = set()
    cur_idx = 0
    '''
    for i in range(idx[0].shape[0]):
        tuple_act = (idx[0][i], idx[3][i])
        if tuple_act not in activation_set:
            test_idx[cur_idx] = tuple_act[0]
            filter_idx[cur_idx] = tuple_act[1]
            cur_idx += 1
            activation_set.add(tuple_act)
            if cur_idx >= TOP_PATCHES:
                break
    '''
    test_idx = idx[0][:TOP_PATCHES]
    width_idx =idx[1][:TOP_PATCHES] * 2
    height_idx = idx[2][:TOP_PATCHES] * 2
    '''
    patches = x_test[test_idx, width_idx: width_idx + np.repeat(12, TOP_PATCHES),
                    height_idx: height_idx + np.repeat(12, TOP_PATCHES), :]
'''

    fig = plt.figure()
    for i in range(TOP_PATCHES):
        ax = fig.add_subplot(4, 3, i + 1)
        ax.set_xticks(())
        ax.set_yticks(())
        m = x_test[test_idx[i], width_idx[i]: width_idx[i] + 12,
                    height_idx[i]: height_idx[i] + 12, :]
        ax.imshow(np.reshape(m, [12, 12]), cmap='Greys_r')
    fig.show()


def one_hot_encode(num_array):
  ohe_array = np.zeros((num_array.size, num_array.max() + 1))
  ohe_array[np.arange(num_array.size), num_array] = 1
  return ohe_array

if __name__ == "__main__":
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
    num_tests, _, _ = test_images.shape
    num_trains, _, _ = train_images.shape

    x_test = np.reshape(test_images[:, :], (num_tests, 28, 28, 1))
    y_test = one_hot_encode(test_labels)

    x_valid = np.reshape(train_images[:VALID_SIZE, :], (VALID_SIZE, 28, 28, 1))
    y_valid = one_hot_encode(train_labels[:VALID_SIZE])

    num_trains -= VALID_SIZE

    x_train = np.reshape(train_images[VALID_SIZE:, :], (num_trains, 28, 28, 1))
    y_train = one_hot_encode(train_labels[VALID_SIZE:])

    print(x_test.shape)
    print(y_test.shape)

    print(x_train.shape)
    print(y_train.shape)
    print(x_valid.shape)
    print(y_valid.shape)

(10000, 28, 28, 1)
(10000, 10)
(58000, 28, 28, 1)
(58000, 10)
(2000, 28, 28, 1)
(2000, 10)


In [ ]:
    tf.compat.v1.disable_eager_execution()


    # Set up placeholders
    x = tf.compat.v1.placeholder(tf.float32, shape=[None, 28, 28, 1])
    y_ = tf.compat.v1.placeholder(tf.float32, shape=[None, 10])
    # Make the fully connected layer
    # (28 - 12) / 2  + 1 = 9 is a size of matrix.
    W_conv1 = weight_variable([12, 12, 1, 25])
    b_conv1 = bias_variable([25])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    h_conv1 = tf.nn.relu(conv2dl1(x_image, W_conv1) + b_conv1)
    print("h_conv1", h_conv1)
    # Make second fully connected layer
    W_conv2 = weight_variable([5, 5, 25, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2dl2(h_conv1, W_conv2) + b_conv2)
    print("h_conv2", h_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    print("h_pool2", h_pool2)
    # 9 / 2 but same -> 10 / 2 = 5, pool operation reduces dimension.
    h_pool2_flat = tf.reshape(h_pool2, [-1, 5 * 5 * 64])
    print("h_pool2_flat", h_pool2_flat)
    W_fc1 = weight_variable([5 * 5 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    print("h_fc1", h_fc1)
    keep_prob = tf.compat.v1.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, rate=1 - (keep_prob))

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    print("y_conv", y_conv)
    print("y_", y_)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.compat.v1.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    train_accs = np.zeros(100)
    valid_accs = np.zeros(100)

h_conv1 Tensor("Relu:0", shape=(None, 14, 14, 25), dtype=float32)
h_conv2 Tensor("Relu_1:0", shape=(None, 14, 14, 64), dtype=float32)
h_pool2 Tensor("MaxPool2d:0", shape=(None, 7, 7, 64), dtype=float32)
h_pool2_flat Tensor("Reshape_1:0", shape=(None, 1600), dtype=float32)
h_fc1 Tensor("Relu_2:0", shape=(None, 1024), dtype=float32)
y_conv Tensor("add_3:0", shape=(None, 10), dtype=float32)
y_ Tensor("Placeholder_1:0", shape=(None, 10), dtype=float32)


In [ ]:
    #x_valid, y_valid = get_tests(TEST_SIZE, false)
    with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        for i in range(NUM_BATCHES):
            batch_x = x_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE, :, :, :]
            batch_y = y_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE, :]
            print(batch_x.shape)
            print(batch_y.shape)
            H1 = sess.run(h_conv1, feed_dict={x:batch_x, y_: batch_y})
            print(H1.shape)
            H2 = sess.run(h_conv2, feed_dict={x:H1})
            print(H2.shape)
            sess.run(train_step, feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})
            if i % TEST_ID == TEST_ID - 1:
                print("Finished processing ", i + 1, " batches.")
                train_accs[i // TEST_ID] = sess.run(accuracy, feed_dict={x: batch_x,
                                                                         y_: batch_y, keep_prob: 1.0})
                valid_accs[i // TEST_ID] = sess.run(accuracy, feed_dict={x: x_valid,
                                                                         y_: y_valid, keep_prob: 1.0})
                print('Step {}, training accuracy {:.3f}'.format(i + 1, train_accs[i // TEST_ID]))
                print('Step {}, validation accuracy {:.3f}'.format(i + 1, valid_accs[i // TEST_ID]))
        #x_test, y_test = get_tests(10000, False)
        print(sess.run(accuracy, feed_dict={x: x_test, y_: y_test, keep_prob: 1.0}))
        W = W_conv1.eval(sess)
        visualize_layer(W)
        H = sess.run(h_conv1, feed_dict={x: x_test})[:, :, :, :VISUAL_FILTERS]
        visualize_patches(H, x_test)
        t=1

Device mapping: no known devices.
(50, 28, 28, 1)
(50, 10)
(50, 14, 14, 25)


ValueError: Cannot feed value of shape (50, 14, 14, 25) for Tensor Placeholder:0, which has shape (None, 28, 28, 1)

In [ ]:
!tf_upgrade_v2 --intree test_v1/ --outtree test_v2 --reportfile report.txt